## Notebook to use enron email data from Kaggle to search using keyword, semantic search and vector search

### Load and Parse Data

In [ ]:
import os, sys, email
import numpy as np 
import pandas as pd
from tqdm import tqdm

In [ ]:
# Load the CSV file (only 1000 rows for now)
file_path = 'enronemails.csv'
emails_df = pd.read_csv(file_path, nrows=1000)

In [ ]:
# Function to parse the email content
def parse_email(message):
    from email.parser import Parser

    email = Parser().parsestr(message)
    return {
        "From": email.get('From'),
        "To": email.get('To'),
        "Subject": email.get('Subject'),
        "Date": email.get('Date'),
        "Body": email.get_payload()
    }

In [ ]:
# Apply the function to each row in the dataframe
parsed_emails = emails_df['message'].apply(parse_email)
parsed_emails_df = pd.DataFrame(parsed_emails.tolist())

# Merge the parsed email details with the original dataframe
emails_df = emails_df.join(parsed_emails_df)

# Display the first few rows of the updated dataframe
emails_df.head()

In [ ]:
#Remove rows with no subject or 'Test' subject
emails_df = emails_df[emails_df['Subject'] != '']
emails_df = emails_df[emails_df['Subject'] != 'test']
print(len(emails_df))

### -> Keyword Search

In [ ]:
from typing import Dict, List

def search_email_chain_by_keyword(dataframe: pd.DataFrame, keyword: str) -> List[Dict]:
    """
    Searches for a keyword in the email body and returns a chronological list of
    people who used the keyword throughout an email chain.

    Args:
    dataframe (pd.DataFrame): The dataframe containing the email data.
    keyword (str): The keyword to search for.

    Returns:
    List[Dict]: A chronological list of people who used the keyword in the email chain.
    """
    # Filter the dataframe for rows containing the keyword in the Body
    keyword_filtered = dataframe[dataframe['Body'].str.contains(keyword, case=False, na=False)]

    # Sort the result by Date
    keyword_filtered_sorted = keyword_filtered.sort_values(by='Date')

    # Extract relevant information
    email_chain = []
    for _, row in tqdm(keyword_filtered_sorted.iterrows(), desc="Searching Emails"):
        email_info = {
            "From": row['From'],
            "To": row['To'],
            "Date": row['Date'],
            "Subject": row['Subject'],
            "Body": row['Body']
        }
        email_chain.append(email_info)

    return email_chain

In [ ]:
# Keyword search
search_results_chain = search_email_chain_by_keyword(emails_df, "Looking for market trends")
search_results_chain[:5] 

### Semantic Search using NLTK

#### Helper Functions

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

def preprocess(text):
    # Preprocess and tokenize emails and query
    stop_words = set(stopwords.words('english'))
    
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalpha() and word not in stop_words]

#### -> Semantic Search

In [ ]:
def semantic_search_email_body(dataframe, query, top_n=5):
    processed_emails = [' '.join(preprocess(email)) for email in tqdm(dataframe['Body'].fillna(""), desc="Processing emails")]
    processed_query = ' '.join(preprocess(query))

    # Create TF-IDF vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_emails + [processed_query])

    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

    # Get top N similar emails
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]

    # Return the top N similar emails
    return dataframe.iloc[top_indices]

In [ ]:
pd.set_option('display.max_colwidth', None)

# Example usage
search_results_semantic = semantic_search_email_body(emails_df, "Looking for market trends", top_n=3)
search_results_semantic['Body']

In [ ]:
# Example usage
search_results_semantic = semantic_search_email_body(emails_df, "Intellectual Prpperty", top_n=3)
search_results_semantic['Body']

### Search by using embeddings created by Open AI. Embeddings are stored in Pgvector.

#### Helper Functions for Tokenization

In [ ]:
import tiktoken

# Helper function: calculate cost of embedding num_tokens
# Assumes we're using the text-embedding-ada-002 model
# See https://openai.com/pricing
def get_embedding_cost(num_tokens):
    return num_tokens/1000*0.0001
    
# Helper function: calculate total cost of embedding all content in the dataframe
def get_total_embeddings_cost():
    total_tokens = 0
    for i in range(len(emails_df.index)):
        text = emails_df['Body'][i]
        token_len = num_tokens_from_string(text)
        total_tokens = total_tokens + token_len
    total_cost = get_embedding_cost(total_tokens)
    return total_cost

# Helper func: calculate number of tokens
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
#Since emails are deleted, lets recreate the Index to avoid index error
emails_df.reset_index(drop=True, inplace=True)

In [ ]:
# quick check on total token amount for price estimation
total_cost = get_total_embeddings_cost()
print("estimated price to embed this content = $" + str(total_cost))

#### Create Embeddings

In [ ]:
from openai import OpenAI
import os

# Get openAI api key by reading local .env file
from dotenv import load_dotenv
load_dotenv('../.env')
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

embeddings = []
def get_embedding(text, model="text-embedding-ada-002", max_length=2048):
    text = text.replace("\n", " ")
    
    #Split the text into chunks based on max token length
    chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]

    for chunk in chunks:
        try:
            # Make the API call to get embeddings for the chunk        
            response = client.embeddings.create(
                input=chunk, 
                model=model
            )
            embeddings.append(response.data[0].embedding)
        except OpenAI.error.OpenAIError as e:
            print(f"An error occurred: {e}")    
    
    return embeddings

# text = 'Here is our forecast' 
# embedding = get_embedding(text)
# embedding

In [ ]:
# Create embeddings for each piece of content
tqdm.pandas(desc="Processing")

emb_values = []
for body in tqdm(emails_df['Body'], desc="Processing rows"):
    try:
        emb_values.append(get_embedding(body))
    # except BadRequestError as e:
    #     emb_values.append('')
    #     # Handle the error, e.g., by reducing the prompt size and retrying
    #     print("The request was too long:", e)
    except Exception as e:
        emb_values.append('')
        print("Error:", e)

# Assigning the results back to the new column
emails_df['Embeddings'] = emb_values

#### Store Embeddings and Email data in Pgvector

In [ ]:
# Create a new dataframe from the list
interested_columns_df = emails_df[['From', 'To', 'Subject', 'Date', 'Body', 'Embeddings']]

In [ ]:
len(interested_columns_df)

In [ ]:
# Connect and configure pgvector
connection_string  = os.environ['LOCAL_POSTGRESS_CONNECTION_STRING']

In [ ]:
import pgvector
import math
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

In [ ]:
# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(connection_string)
cur = conn.cursor()

In [ ]:
#install pgvector 
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

# Register the vector type with psycopg2
register_vector(conn)

# Create table to store embeddings and metadata
table_create_command = """
CREATE TABLE IF NOT EXISTS enron_embeddings (
            id bigserial primary key, 
            EmailFrom text,
            EmailTo text,
            EmailSubject text,
            EmailDate date,
            EmailBody text,
            Embedding vector(1536)
            );
            """

cur.execute(table_create_command)
cur.close()
conn.commit()

In [ ]:
register_vector(conn)
cur = conn.cursor()

In [ ]:
from datetime import datetime
from psycopg2.extras import execute_values
from email.utils import parsedate_tz, mktime_tz
from tqdm import tqdm

def batch_insert_generator(df, batch_size=100):
    batch = []
    for index, row in df.iterrows():
        batch.append((
            row['From'],
            row['To'],
            row['Subject'],
            str(datetime.fromtimestamp(mktime_tz(parsedate_tz(row['Date'])))),
            row['Body'],
            row['Embeddings'].tolist() if isinstance(row['Embeddings'], np.ndarray) else row['Embeddings']
        ))
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

# Define batch size
BATCH_SIZE = 100

# Use tqdm to show progress bar
for batch in tqdm(batch_insert_generator(interested_columns_df, BATCH_SIZE), desc="Inserting batches"):
    execute_values(cur, """
    INSERT INTO enron_embeddings (EmailFrom, EmailTo, EmailSubject, EmailDate, EmailBody, Embedding)
    VALUES %s
    """, batch)
    conn.commit()  # Commit after each batch

In [ ]:
from datetime import datetime
from email.utils import parsedate_tz, mktime_tz

#Batch insert embeddings and metadata from dataframe into PostgreSQL database
data_list = [(row['From'], row['To'], row['Subject'], str(datetime.fromtimestamp(mktime_tz(parsedate_tz(row['Date'])))), 
              row['Body'], 
              row['Embeddings'].tolist() if isinstance(row['Embeddings'], np.ndarray) else row['Embeddings']) 
             for index, row in tqdm(interested_columns_df.iterrows(), total=interested_columns_df.shape[0], desc="Inserting rows")]

# Use execute_values to perform batch insertion
execute_values(cur, "INSERT INTO enron_embeddings (EmailFrom, EmailTo, EmailSubject, EmailDate, EmailBody, Embedding) VALUES %s", data_list)
# Commit after we insert all embeddings
conn.commit()

In [ ]:
cur.execute("SELECT COUNT(*) as cnt FROM enron_embeddings;")
num_records = cur.fetchone()[0]
print("Number of vector records in table: ", num_records,"\n")

In [ ]:
# Create an index on the data for faster retrieval

#calculate the index parameters according to best practices
num_lists = num_records / 1000
if num_lists < 10:
    num_lists = 10
if num_records > 1000000:
    num_lists = math.sqrt(num_records)

#use the cosine distance measure, which is what we'll later use for querying
cur.execute(f'CREATE INDEX ON enron_embeddings USING ivfflat (Embedding vector_cosine_ops) WITH (lists = {num_lists});')
conn.commit() 

#### -> Search Pgvector using KNN operator

In [ ]:
# Helper function: Get top 3 most similar documents from the database
def get_top3_similar_emails(query_embedding, conn):
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    cur = conn.cursor()
    # Get the top 3 most similar documents using the KNN <=> operator
    #To_CHAR(EmailDate, 'YY/MM/DD'), 
    cur.execute("SELECT EmailFrom, EmailTo, EmailDate, EmailSubject, EmailBody FROM enron_embeddings ORDER BY Embedding <=> %s LIMIT 3", (embedding_array,))
    top3_docs = cur.fetchall()
    return top3_docs

In [ ]:
user_input = 'Looking for market trends'
related_emails = get_top3_similar_emails(get_embedding(user_input), conn)
for item in related_emails:
    print(f"-->{item[4]}")

In [ ]:
user_input = 'Payment'
related_emails = get_top3_similar_emails(get_embedding(user_input), conn)
for item in related_emails:
    print(f"-->{item[4]}")

#### Helper functions to save in json..

In [ ]:
# # Keys for the JSON objects
# import json
# keys = ['from', 'to', 'title', 'content']

# # Convert each tuple to a dictionary and add it to the list
# json_list = [dict(zip(keys, t)) for t in related_emails]

# json_str = json.dumps(json_list, indent=4)

# # Save the JSON string to a file
# with open('filtered_emails.json', 'w') as file:
#     file.write(json_str)


In [ ]:
# Define the column names to create dataframe for
column_names = ['from', 'to', 'date', 'title', 'content']

# Create the DataFrame
df = pd.DataFrame(related_emails, columns=column_names)

random_integers = np.random.randint(1, 100, size=len(df))
#insert type and id in the begining
df.insert(0, 'type', 'email')
df.insert(1, 'id', random_integers)

# Convert the datetime.date objects to strings
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
import json
# Convert the DataFrame to a list of dictionaries
json_list = df.to_dict(orient='records')
json_str = json.dumps(json_list, indent=4)

print(json_str)

In [ ]:
# Save the JSON string to a file
with open('filtered_emails.json', 'w') as file:
    file.write(json_str)